# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [2]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('../data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [3]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [4]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'full','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()

    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T
    df_api.columns = [c.split('. ')[1] for c in df_api.columns] 
    df_api = df_api.reset_index().rename(columns={'index':'date'})
    df_api['date'] = pd.to_datetime(df_api['date'])
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df_api[col] = pd.to_numeric(df_api[col])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()
    df_api = df_api.rename(columns={'Date':'date','Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'})

required_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
v_api = validate(df_api, required_cols)
print("API Validation Results:", v_api)

if v_api['missing']:
    print("Validation FAILED: Missing required columns:", v_api['missing'])
elif v_api['na_total'] > 0:
    print(f"Validation WARNING: Data contains {v_api['na_total']} null values.")
elif v_api['shape'][0] < 1000:
    print(f"Validation WARNING: Low row count ({v_api['shape'][0]}) for 'full' data size.")
else:
    print("Validation Passed: Data appears to be complete and valid.")

API Validation Results: {'missing': [], 'shape': (6488, 6), 'na_total': 0}
Validation Passed: Data appears to be complete and valid.


In [5]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved ../data/raw/api_source-alpha_symbol-AAPL_20250819-101945.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [6]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')

required_cols = ['Symbol', 'Security']
v_scrape = validate(df_scrape, required_cols)
print("Scrape Validation Results:", v_scrape)

rows = v_scrape['shape'][0]
if v_scrape['missing']:
    print("Validation FAILED: Missing required columns:", v_scrape['missing'])
elif not (490 < rows < 510): 
    print(f"Validation WARNING: Row count ({rows}) is outside the expected range of 490-510.")
else:
    print("Validation Passed: Data appears to be complete and valid.")


Scrape Validation Results: {'missing': [], 'shape': (880, 8), 'na_total': 758}
Validation WARNING: Row count (880) is outside the expected range of 490-510.


In [7]:
_ = save_csv(df_scrape, prefix='scrape', site='wikipedia', table='SP500-List')

Saved ../data/raw/scrape_site-wikipedia_table-SP500-List_20250819-101945.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.

# **Work Summary**

This script ingests data from two distinct sources: stock prices via an API and a list of companies via web scraping.

#### 1. Alpha Vantage API (Stock Prices)
* **Data Source**: `Alpha Vantage`
* **Parameters**: This process uses `function: TIME_SERIES_DAILY`, `symbol: "AAPL"`, an API key loaded from the `.env` file, and `outputsize: full` to fetch the complete historical data.
* **Validation Logic**: The script validates the data by checking if the DataFrame is not empty, contains all required columns, has more than 1000 rows, and is free of null values.

#### 2. Wikipedia Web Scrape (S&P 500 Companies List)
* **Data Source/URL**: `https://en.wikipedia.org/wiki/List_of_S%26P_500_companies`
* **Parameters**: No specific parameters are used; the script simply fetches the first table on the page.
* **Validation Logic**: It validates the data by checking if the DataFrame is not empty, contains the `'Symbol'` and `'Security'` columns, and has a row count between 490 and 510.

---

### Regarding the .env File

The `python-dotenv` library is used to load the `ALPHAVANTAGE_API_KEY` from the `.env` file. The `.env` file **has been added** to `.gitignore`, and an `.env.example` will be committed as a placeholder.

---

### Assumptions & Risks

* **Assumptions**

    * **API Structure Stability**: Assumes the JSON structure (e.g., the key `'Time Series (Daily)'`) and column names returned by the Alpha Vantage API will remain consistent.

    * **Web Page Structure Stability**: Assumes the target table on the S&P 500 list Wikipedia page will always be the first `<table>` element.

    * **Data History Depth**: Assumes that the daily data for a mature stock (like AAPL) should exceed 1000 records.

    * **Number of S&P 500 Constituents**: Assumes the number of constituents in the S&P 500 index is between 490 and 510.

* **Risks**

    * **API Key Exposure**: The API key will be exposed if the `.env` file is accidentally committed to the code repository.

    * **Upstream Data Source Changes**: The structure of the API or the Wikipedia page could change at any time, which would cause the script to fail or extract incorrect data.

    * **API Limitations**: AlphaVantage might adjust its data access limitations; for instance, requiring a subscription to access `TIME_SERIES_DAILY_ADJUSTED` data.